In [1]:
# workaround for incredibly stupid vscode on snap bug
from os import environ
krb5ccname = environ["KRB5CCNAME"]
if "hostfs" in krb5ccname:
    krb5ccname = krb5ccname.replace("/var/lib/snapd/hostfs", "")
    environ["KRB5CCNAME"] = krb5ccname

In [2]:
import ROOT
from analysis_framework import Dataset
from ReweightingHelper import ReweightingHelper
from AltSetupHandler import AltSetupHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x9760e40


In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad
n_threads = 12
prod = False
# prod = True
no_rvec = True
write_outputs = False
# write_outputs = True
# plot_dir_postfix = "-new-cuts"
dataset_path = "data/datasets/selected-objects/test.json"
# output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/selected-objects/test"
# output_meta_path = "data/datasets/selected-objects"
# output_meta = f"{output_meta_path}/test.json"
# checked_output_meta = f"{output_meta_path}/checked-test.json"
# output_collections = [
#     "true_lep_lvec", "true_nu_lvec", "true_quark1_lvec", "true_quark2_lvec",
#     "iso_lep_lvec", "nu_lvec", "R2Jet_sel1_lvec", "R2Jet_sel2_lvec",
#     ]
# true lvecs do not exist in every df so cannot be explicitly requested...
# urgh but empty snapshots are also not allowed
output_collections = r"(true_\w+_lvec)|(iso_lep_lvec)|(nu_lvec)|(R2Jet_sel1_lvec)|(R2Jet_sel2_lvec)"
# plot_dir = f"plots/pre-selection/test{plot_dir_postfix}"
if prod:
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs-min-aa-min-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/min-higgs.json"
    dataset_path = "data/datasets/selected-objects/signal-only.json"
    # output_path = "root://eospublic.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/min-higgs-d"
    # output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/selected-objects/signal-only"
    # output_meta_path = "data/datasets/selected-objects"
    # output_meta = f"{output_meta_path}/signal-only.json"
    # checked_output_meta = f"{output_meta_path}/checked-signal-only.json"
    # plot_dir = "plots/pre-selection/full"
    # plot_dir = f"plots/pre-selection/min-higgs{plot_dir_postfix}"


In [4]:
# ROOT.EnableImplicitMT(n_threads)

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = ReweightingHelper(dataset)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xd80e000


In [7]:
alt_setup_handler = AltSetupHandler("""
{
  "SM": {
    "g1z": 1.0,
    "ka": 1.0,
    "la": 0.0
  },
"variations": [
    0.001,
    0.0001,
    1e-08
  ]
}
""", mirror=False, combinations=False)
alt_configs = alt_setup_handler.get_alt_setup()
print(alt_configs)
analysis.initialise_omega_wrappers(alt_configs)

{'g1z_pos_1em03': {'g1z': 1.001, 'ka': 1.0, 'la': 0.0}, 'ka_pos_1em03': {'g1z': 1.0, 'ka': 1.001, 'la': 0.0}, 'la_pos_1em03': {'g1z': 1.0, 'ka': 1.0, 'la': 0.001}, 'g1z_pos_1em04': {'g1z': 1.0001, 'ka': 1.0, 'la': 0.0}, 'ka_pos_1em04': {'g1z': 1.0, 'ka': 1.0001, 'la': 0.0}, 'la_pos_1em04': {'g1z': 1.0, 'ka': 1.0, 'la': 0.0001}, 'g1z_pos_1em08': {'g1z': 1.00000001, 'ka': 1.0, 'la': 0.0}, 'ka_pos_1em08': {'g1z': 1.0, 'ka': 1.00000001, 'la': 0.0}, 'la_pos_1em08': {'g1z': 1.0, 'ka': 1.0, 'la': 1e-08}}


In [8]:
analysis.init_categories()
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]

True


In [9]:
analysis.book_weights(signal_category,
                      [
                          "true_beam_e_lvec",
                          "true_beam_p_lvec",
                          "true_lep_lvec",
                          "true_nu_lvec",
                          "true_quark1_lvec",
                          "true_quark2_lvec"
                      ]
                     )

In [10]:
if write_outputs:
    analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [11]:
%%time
analysis.run()

CPU times: user 74 ms, sys: 1.81 ms, total: 75.8 ms
Wall time: 77.5 ms


Warning in <RunGraphs>: Got an empty list of handles, now quitting.


In [12]:
# if write_outputs:
    # analysis.check_snapshots("events", output_path, checked_output_meta)